# SL Fidelidad - 02_Join Data

En esta notebook se analizan los tres dataframe generados y luego se los use en el df final.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import pickle

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
# seteo carpetas
BASE = Path().absolute().parent
RAW = Path(BASE/'raw')
DATA = Path(BASE/'data')

## Bases

- Socios: df_socios
- Abonos: df_abonos
- Eventos: df_eventos

In [ ]:
# socios
df_socios = pickle.load( open( DATA/"df_socios.p", "rb" ) )
# abonos
df_abonos = pickle.load( open( DATA/"df_abonos.p", "rb" ) )
# eventos
df_eventos = pickle.load( open( DATA/"df_eventos.p", "rb" ) )

## Exploro bases

In [ ]:
# variables
print(f'variables de df_socios: \n{df_socios.dtypes}\n')
print(f'variables de df_abonos: \n{df_abonos.dtypes}\n')
print(f'variables de df_eventos: \n{df_eventos.dtypes}\n')

In [ ]:
# observaciones
print(df_socios.shape)
print(df_abonos.shape)
print(df_eventos.shape)

In [ ]:
# analisis de integridad
print(f'hay {df_abonos.shape[0] - sum([(i in df_socios.index) for i in df_abonos.index])} que estan en abonos y no en socios')
print(f'hay {df_eventos.shape[0] - sum([(i in df_socios.index) for i in df_eventos.index])} que estan en eventos y no en socios')

In [ ]:
# analizo los que estan en eventos y no en socios
eventos_socios = [(i, i in df_socios.index) for i in df_eventos.index]
eventos_nosoc = []
for i in eventos_socios:
    if i[1] == False:
        eventos_nosoc.append(i[0])
# eventos_nosoc

## Junto las bases y creo algunas variables adicionales

In [ ]:
# junto las  tres tablas
df = pd.merge( left=pd.merge(df_socios, df_abonos, how='left', left_index=True, right_index=True),
                    right=df_eventos, how='left', left_index=True, right_index=True)
df.columns

In [ ]:
# reemplazo los NaN por missing cuando corresponde
varstofill = ['abo_bid_2020', 'abo_bid_2021', 'abo_pol_2020', 'abo_pol_2021', 'eve_2018', 'eve_2019', 'eve_2020']

for var in varstofill:
    df[var] = df[var].fillna(0)

In [ ]:
# creo variable partidos
df['eve'] = df['eve_2018'] + df['eve_2019'] + df['eve_2020']
df.loc[df['eve']==0, 'partidos'] = "Ninguno"
df.loc[(df['eve']>=1) & (df['eve']<5), 'partidos'] = "Entre 1 y 4"
df.loc[(df['eve']>=5) & (df['eve']<10), 'partidos'] = "Entre 5 y 9"
df.loc[df['eve']>=10, 'partidos'] = "Mas de 10"

In [ ]:
# casteo variables fecha
df['fecha_ingreso'] = pd.to_datetime(df['fecha_ingreso'], format='%d/%m/%Y')
df['fecha_nac'] = pd.to_datetime(df['fecha_nac'], format='%d/%m/%Y')

In [ ]:
# edad
df['f_ingreso_18'] = [x + relativedelta(years=18)  for x in df['fecha_nac']]
df['f_ingreso_18'] = np.where(df['fecha_ingreso']<df['f_ingreso_18'], df['f_ingreso_18'], df['fecha_ingreso'])
df['antig'] = [round((datetime.now() - x).days/365) for x in df['f_ingreso_18']]
df['edad'] = [np.nan if pd.isnull(x) else round((datetime.now() - x).days/365) for x in df['fecha_nac']]

In [ ]:
df.loc[(df['edad']<18), 'grupo_etario'] = "Menores a 18"
df.loc[(df['edad']>=18) & (df['edad']<=29), 'grupo_etario'] = "Entre 18 y 29"
df.loc[(df['edad']>=30) & (df['edad']<=44), 'grupo_etario'] = "Entre 30 y 44"
df.loc[(df['edad']>=45) & (df['edad']<=59), 'grupo_etario'] = "Entre 45 y 59"
df.loc[df['edad']>=60, 'grupo_etario'] = "Mayor a 60"

df['grupo_etario'].value_counts()

In [ ]:
# filtro mayores de 70 para el analisis
print(df.shape[0])
df = df[df.edad <= 70]
print(df.shape[0])

In [ ]:
# seteo debito a vitalicios
df.loc[(df['tipo_socio']=='VITALICIO'), 'forma_pago'] = "DEBITO"

## Guardo base final

In [ ]:
pickle.dump( df, open( DATA/"df_fide.p", "wb" ) )